# Fit stochastic SIR models to synthetic data

The Starsim framework includes a built-in susceptible-infected-recovered (SIR) within-host progression module that can be used as a building block to developing more realistic agent-based models. Here, we use that SIR within-host module in combination with mixing pool transmission to create a simple SIR disease model. 

Then, we validate an optimization-based (Optuna) calibration workflow by fitting the model to synthetic data generated from the same model using known parameter values. Re-identifying the known parameter values from synthetic data provides reassurance that the calibration workflow is functioning as expected. For a more realistic Starsim calibration example, please see [Calibration](workflows_calibration.ipynb).

[TODO]: <> (Add link to ModelingHub deterministic example once the URL is live.)

This notebook includes the following two examples:
1. [Agent-based SIR using Optuna](#ex1): Calibrate a stochastic agent-based SIR model implemented using Starsim to data.
2. [Agent-based SIR using Starsim-Optuna integration](#ex2): Use the Calibration class built into the Starsim framework to calibrate a stochastic agent-based SIR model implemented using Starsim to data.

In [ ]:
# Configure notebook autoreloading and inline plotting
%load_ext autoreload
%autoreload 2
%matplotlib inline

##%% Imports and settings
import optuna
optuna.logging.set_verbosity(optuna.logging.CRITICAL)
import starsim as ss
import numpy as np
import pandas as pd
import sciris as sc
import matplotlib.dates as mdates
import seaborn as sns
from IPython.display import display

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=optuna.exceptions.ExperimentalWarning)

### Stochastic SIR using Optuna outside of Starsim
<a id="ex1"></a>

First, we create a simple agent-based SIR simulation using Starsim.

In [ ]:
def make_sim():
    """
    Make a simple SIR simulation using Starsim.

    Returns
    -------
    Sim
        A single simulation that has been configured, but not initialized or run.
    """
    sir_pars = dict(
        init_prev = 0.01,
        p_death = 0,
    )
    sir = ss.SIR(pars=sir_pars)

    net = ss.MixingPool(
        beta = 1, # This is a multiplier on the disease beta
        n_contacts = 1,
    )

    sim_pars = dict(
        n_agents = 1000,
        dt = ss.days(1),
        start = '2025-01-01',
        dur = 100,
        verbose = 0,
    )
    sim = ss.Sim(pars=sim_pars, diseases=sir, networks=net)
    return sim

def modify_sim(sim, calib_pars, rand_seed=0):
    """
    Modify the given simulation with the calibration parameters and random seed.
    
    Parameters
    ----------
    sim : Sim
        The simulation to modify.
    calib_pars : dict
        The calibration parameters to apply; note that the parameter values to use are stored in "value."
    rand_seed : int
        The random seed to use for the simulation.

    Returns
    -------
    Sim
        The modified simulation.
    """

    # Explicitly look for each of the calibration parameters and set the appropriate values
    if 'beta' in calib_pars:
        β = ss.perday(calib_pars['beta']['value'])
        sim.pars.diseases.pars['beta'] = β

    if 'gamma' in calib_pars:
        γ = calib_pars['gamma']['value']
        sim.pars.diseases.pars['dur_inf'].set(1/γ)

    sim.pars['rand_seed'] = rand_seed

    return sim


def run_starsim(pars, rand_seed=0):
    """
    Run a Starsim SIR model with given parameters and random seed, returning results.
    
    Parameters
    ----------
    pars : dict
        The parameters dictionary with Optuna a values stored in "value."
    rand_seed : int
        The random seed to use for the simulation.

    Returns
    -------
    dataframe
        The results of the SIR model with columns of S, I, and R and index of Time.
    """
    # Make and modify the simulation
    sim = make_sim()
    sim = modify_sim(sim, pars, rand_seed)

    sim.run() # Run the simulation

    # Extract the results
    results = pd.DataFrame(dict(
        S = sim.results.sir.n_susceptible,
        I = sim.results.sir.n_infected,
        R = sim.results.sir.n_recovered,
    ), index=pd.Index(sim.results.timevec, name='Time'))
    results['rand_seed'] = rand_seed # Store the random seed for later reference

    return results

That set the basic machinery of the SIR simulation model. Next, we'll create some synthetic data to use as calibration targets. Because the model is stochastic, we'll average over several replicates.

In [ ]:
n_reps = 10 # Number of repetitions to run

# These are the true parameters the optimizer will later try to identify
true_pars = dict(
    beta = dict(value=0.12),
    gamma = dict(value=0.03),
)

# Run the starsim SIR simulations in parallel. If you need to run in serial, for example when debugging, simply set serial=True
results_list = sc.parallelize(run_starsim, pars=true_pars, iterkwargs=dict(rand_seed=np.arange(n_reps)), serial=False)
results = pd.concat(results_list) # Combine the results into a single DataFrame
ave = results.groupby('Time').mean().drop(columns='rand_seed') # Average the results over the repetitions

# Extract synthetic data for calibration
observation_times = np.array([pd.Timestamp('2025-01-01')+pd.DateOffset(days=d) for d in [20, 40, 80]])
starsim_data = pd.DataFrame({
    'x': ave.loc[observation_times, 'I'].astype(int),
    'n': ave.loc[observation_times].sum(axis=1),
    'Prevalence': ave.loc[observation_times, 'I'] / ave.loc[observation_times].sum(axis=1)
}, index=pd.Index(observation_times, name='t'))

print('Here is the data extracted from the average simulation to be used during calibration:\n')
display(starsim_data)

# Plot the results, vertical dashed lines indicate the observation times where prevalence is measured
df = results.reset_index().melt(id_vars=['Time', 'rand_seed'], value_vars=['S', 'I', 'R'], var_name='State', value_name='Count')
ax = sns.lineplot(data=df, hue='State', x='Time', y='Count', units='rand_seed', estimator=None, alpha=0.5, lw=0.5, legend=False)
sns.lineplot(data=df, hue='State', x='Time', y='Count', errorbar=('pi', 50), ax=ax, legend=True)
ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(ax.xaxis.get_major_locator()))
for ot in observation_times:
    ax.axvline(ot, ls='--', color='black')
ax.scatter(starsim_data.index, starsim_data['x'], marker='o', color='black', label='Observed data');

Now we'll set about trying to calibrate to this data using the [Optuna](optuna.org) hyperparameter optimization library.

In [ ]:
# We will calibrate two parameters, beta and gamma, each allowed to take on values between 0 and 1.
calib_pars = dict(
    beta = {'low': 0, 'high': 1},
    gamma = {'low': 0, 'high': 1},
)

def evaluate(results, calib_data):
    """
    Evaluate the results of a simulation against calibration data.

    Parameters
    ----------
    results: DataFrame
        The results of a simulation.
    calib_data: DataFrame
        The calibration data to compare against.

    Returns
    -------
    float
        The squared error between the simulation results and the calibration data.
    """
    squared_error = 0
    for step, expected in calib_data.iterrows():
        observed = results.loc[step, 'I'] / results.loc[step].sum()
        squared_error += (observed - expected)**2
    return squared_error

def trial_to_pars(trial):
    """
    Map an Optuna trial to a parameter spec.

    Parameters
    ----------
    trial: optuna.Trial
        The trial to map.

    Returns
    -------
    dict
        The parameters dictionary with Optuna a values stored in "value."
    """
    pars = sc.dcp(calib_pars)
    for name, spec in pars.items():
        spec['value'] = trial.suggest_float(name=name, **spec)
    return pars

In [ ]:
def objective(trial):
    """
    Evaluate a trial by running the Starsim model and comparing the results to the calibration data.

    Parameters
    ----------
    trial: optuna.Trial
        The trial to map.

    Returns
    -------
    float
        The objective value of the trial
    """
    pars = trial_to_pars(trial)
    result = run_starsim(pars, rand_seed=0)
    objective = evaluate(result[['S', 'I', 'R']], starsim_data[['Prevalence']])
    return objective

def callback_on_threshold(study, trial):
    # Assuming we want to minimize the objective
    if study.best_value is not None and study.best_value < 1e-2:
        print(f"Objective value threshold reached: {study.best_value}. Stopping study.")
        study.stop()


# The following two lines create an Optuna study and optimize the objective function - easy!
study = optuna.create_study()
study.optimize(objective, n_trials=2500, callbacks=[callback_on_threshold]) # We'll run up to 2,500 total trials. This may take a few minutes, depending on your computer. Consider reducing this number if you like.

# Print a summary
sc.colorize(color='blue', string=f'The best parameters identified by the optimization are:\n\
    * {study.best_params}\n\
    These parameters should be close to the true parameters:\n\
    * {true_pars}\n\
    The best parameters resulted in a loss of {study.best_value}.')

# Plot the results
fig = optuna.visualization.matplotlib.plot_optimization_history(study);
fig.axes.set_yscale('log')

ax = optuna.visualization.matplotlib.plot_contour(study);
ax.axvline(true_pars['beta']['value'], ls='--', color='black')
ax.axhline(true_pars['gamma']['value'], ls='--', color='black')
ax.scatter(study.best_params['beta'], study.best_params['gamma'], 100, marker='x', color='red', zorder=10);

The parameters seem pretty good! Let's see what simulation look like using the best parameters.

In [ ]:
n_reps = 10 # Number of repetitions to run

# These are the true parameters the optimizer will later try to identify.
best_pars = dict(
    beta = dict(value=study.best_params['beta']),
    gamma = dict(value=study.best_params['gamma']),
)

# Run the starsim SIR simulations in parallel. If you need to run in serial, for example when debugging, simply set serial=True
results_list = sc.parallelize(run_starsim, pars=best_pars, iterkwargs=dict(rand_seed=np.arange(n_reps)), serial=False)
results = pd.concat(results_list) # Combine the results into a single DataFrame

# Plot the results, vertical dashed lines indicate the observation times where prevalence is measured
df = results.reset_index().melt(id_vars=['Time', 'rand_seed'], value_vars=['S', 'I', 'R'], var_name='State', value_name='Count')
ax = sns.lineplot(data=df, hue='State', x='Time', y='Count', units='rand_seed', estimator=None, alpha=0.5, lw=0.5, legend=False)
sns.lineplot(data=df, hue='State', x='Time', y='Count', errorbar=('pi', 50), ax=ax, legend=True)
ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(ax.xaxis.get_major_locator()))
for ot in observation_times:
    ax.axvline(ot, ls='--', color='black')
ax.scatter(starsim_data.index, starsim_data['x'], marker='o', color='black', label='Observed data')

The fit looks great! Onto the next example.

### Stochastic SIR using Optuna integrated within Starsim
<a id="ex2"></a>

The previous example used the Optuna Bayesian optimization algorithm as a "wrapper" around a stochastic disease model. Starsim provides built-in integration with Optuna to make advanced model calibration easier. We demonstrate that linkage below.

In this example, we use a Beta-binomial likelihood function; however, results shouldn't be too different from the simple example above that used a squared error metric.

In [ ]:
sim = make_sim() # Begin by making a single "base" simulation with default parameters

# This example will use a single calibration component
# We choose a Beta-binomial functional form to represent the "prevalence survey" data, taking advantage of both the numerator (x) and denominator (n) data.
prevalence_component = ss.BetaBinomial(
    name = 'SIR Disease Prevalence',
    conform = 'step_containing', # The starsim_data data has a date for each observation. The "step_containing" conform method will extract simulation results on the time step containing the observation date.

    # Here is the data we are trying to match, using the "x" and "n" columns from the starsim_data DataFrame.
    expected = starsim_data[['x', 'n']],

    # And here is how we will extract the relevant data from the simulation results
    extract_fn = lambda sim: pd.DataFrame({
        'x': sim.results.sir.n_infected, # Numerator
        'n': sim.results.n_alive, # Denominator
    }, index=pd.Index(sim.results.timevec, name='t')),
)

# Make the calibration
calib = ss.Calibration(
    sim = sim,                              # The base simulation
    calib_pars = calib_pars,                # The calibration parameters
    build_fn = modify_sim,                  # The function to modify the base simulation with the calibration parameters
    reseed = False,                         # In this example, we will not reseed the simulation for each calibration trial
    components = [prevalence_component],    # The calibration components
    total_trials = 500,                     # We will run 500 total trials, as before. This might take some time, depending on your computer.
    verbose = False,                        # Shh...

    sampler = optuna.samplers.TPESampler(n_startup_trials=100)
)

calib.calibrate() # Let's go!

# Print out a summary
sc.colorize(color='blue', string=f'The best parameters identified by the optimization are:\n\
    * {calib.best_pars}\n\
    These parameters should be close to the true parameters:\n\
    * {true_pars}\n\
    The best parameters resulted in a loss of {calib.study.best_value}.')

It seems like the best parameter values are pretty close to the right values, so that's good. We can look at all the results and easily create a DataFrame containing the top K runs.

In [ ]:
df = calib.to_df(top_k=10)
display(df)

In [ ]:
# Plot the results
figs = calib.plot_optuna(['plot_optimization_history', 'plot_contour']);
figs[0].axes.set_yscale('log')

figs[1].axvline(true_pars['beta']['value'], ls='--', color='black');
figs[1].axhline(true_pars['gamma']['value'], ls='--', color='black');
figs[1].scatter(calib.study.best_params['beta'], calib.study.best_params['gamma'], 100, marker='x', color='red', zorder=10);

Finally, let's run some simulations with the best parameters and compare to the calibration data.

In [ ]:
n_reps = 10 # Number of repetitions to run

# These are the true parameters! Shhh, don't tell the optimizer.
best_pars = dict(
    beta = dict(value=calib.study.best_params['beta']),
    gamma = dict(value=calib.study.best_params['gamma']),
)

# Run the starsim SIR simulations in parallel, cool! If you need to run in serial, for example when debugging, simply set serial=True
results_list = sc.parallelize(run_starsim, pars=best_pars, iterkwargs=dict(rand_seed=np.arange(n_reps)), serial=False)
results = pd.concat(results_list) # Combine the results into a single DataFrame

# Plot the results, vertical dashed lines indicate the observation times where prevalence is measured
df = results.reset_index().melt(id_vars=['Time', 'rand_seed'], value_vars=['S', 'I', 'R'], var_name='State', value_name='Count')
ax = sns.lineplot(data=df, hue='State', x='Time', y='Count', units='rand_seed', estimator=None, alpha=0.5, lw=0.5, legend=False)
sns.lineplot(data=df, hue='State', x='Time', y='Count', errorbar=('pi', 50), ax=ax, legend=True)
ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(ax.xaxis.get_major_locator()))
for ot in observation_times:
    ax.axvline(ot, ls='--', color='black')
ax.scatter(starsim_data.index, starsim_data['x'], marker='o', color='black', label='Observed data');

These results also look great.